In [3]:
"""
List of Functions:
  timenow()
  show_file_contents(filename)
  prep_temperature_data()
  plot_temperature_timecourse(setpoint)
  """

import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import numpy
import os
from matplotlib import cm
matplotlib.rc('font', family='DejaVu Sans')

import warnings
warnings.filterwarnings("ignore", module="numpy")
warnings.filterwarnings("ignore", module="matplotlib")

plot_each_spectrum = True
plot_each_lissajous = False

expname = '2016-12-13_19:55:12'
time_micro = '2016-11-17_13:52:49'
time_sys = '2016-11-17_13:47:40'

tmax = 13000
nom_v = 100
nom_f = 60
nom_q = 21

# absolute path to the main project folder
droot = os.path.abspath(os.path.join(os.sep,'mnt','storage'))
# relative paths to the directores we'll be using
ddata = os.path.join(droot,'data')
dexp = os.path.join(ddata,expname)
din = os.path.join(dexp,'in')
dout = os.path.join(dexp,'out')
dplot = os.path.join(dexp,'plot')
dbin = os.path.join(dexp,'bin')
dintemp = os.path.join(din, 'temperature')
dinthermo = os.path.join(din, 'thermograph')
dinosc = os.path.join(din, 'oscilloscope')
dinspectra = os.path.join(din, 'spectra')
dinintense = os.path.join(din, 'photointensity')
dincurrent = os.path.join(din, 'current')

# The experiment started at the folder timestamp
starttime = datetime.strptime(expname,"%Y-%m-%d_%H:%M:%S")
timepoint_micro = datetime.strptime(time_micro,"%Y-%m-%d_%H:%M:%S")
timepoint_system = datetime.strptime(time_sys,"%Y-%m-%d_%H:%M:%S")
# adjust the Arduino timestamps to match the system time (with a manual correction)
microtime_correction = timepoint_system - timepoint_micro - timedelta(seconds=1)

In [4]:
def timenow():
    print('The time now is: {:%Y-%m-%d_%H:%M:%S}'.format(datetime.now()))

def show_file_contents(filename):
    # show the user the results of the processing job
    print("Now processing data file: {:s}".format(filename))
    print("The contents of the file look like: ")
    with open(filename,'r') as myfile:
        for i in range(0,5):
            print('    {:s}'.format(myfile.readline()))

def prep_arduino_data():
    ### WORKS GOOD IN PYTHON3 (BC 2016-10-01)
    frawtemp = os.path.join(dintemp,os.listdir(dintemp)[0])
    fcleantemp = os.path.join(dout,'time_temperature.csv')
    fcleanset = os.path.join(dout,'time_setpoint.csv')
    
    # show the user information about the file being processed
    timenow()
    print("Building temperature timecourse...")
    show_file_contents(frawtemp) 
    
    ts2sec = lambda x: (datetime.strptime(x.decode(), '%Y-%m-%d_%H:%M:%S')-starttime+microtime_correction).total_seconds()
    
    col_names = ('dt','ms','ctrl',
             'bright','set','ctrl_output',
             'intensity','ttube','tsurf',
             'setv','setf','setq',
             'outv','outf','outq',
             'measv','measf','measq',
             'tamb','none')

    col_types = (datetime,int,bool,
              float,float,float,
              float,float,float,
              float,float,float,
              float,float,float,
              float,float,float,
              float)
    
    dataset = numpy.genfromtxt(frawtemp,delimiter=',',invalid_raise=False,
                               usecols=[0,1,9,10,11,8,7,6],skip_header=60,
                               dtype=col_types,names=col_names,converters={'Time':ts2sec})
    time_us=[]
    # correcting the timestamps for millisecond resolution
    now_seconds = 0
    prev_seconds = 0
    now_ms = 0
    prev_ms = 0
    ts_ms = 0
    for line in dataset:
        now_seconds = line[0]
        now_ms = line[1]
        if now_seconds != prev_seconds:
            prev_seconds = now_seconds
            prev_ms = now_ms
            ts_ms = 0
        else:
            ts_ms = prev_ms + (now_ms - prev_ms) % 1000
            prev_ms = ts_ms
        time_us.append(now_seconds + ts_ms/1000)
    # reshape and combine the array - time
    time_reshaped = numpy.reshape(time_us,(len(time_us),1))
    
    # reshape and combine the arrays - time / temperature
    tsurface_reshaped = numpy.reshape(dataset['tsurf'],(len(dataset['tsurf']),1))
    ttube_reshaped = numpy.reshape(dataset['ttube'],(len(dataset['ttube']),1))
    dataset_clean = numpy.concatenate((time_reshaped,tsurface_reshaped,ttube_reshaped),axis=1)
    # save to file - time / temperature
    numpy.savetxt(fcleantemp,dataset_clean,fmt='%f,%f,%f')
    # show the user the results of the processing job - time / temperature
    show_file_contents(fcleantemp)
    timenow()
    print("Temperature timecourse complete!\n\n")
    
    # reshape and combine the arrays - time / setpoint
    print("Building setpoint timecourse...")
    set_v_reshaped = numpy.reshape(dataset['setv'],(len(dataset['setv']),1))
    set_f_reshaped = numpy.reshape(dataset['setf'],(len(dataset['setf']),1))
    set_q_reshaped = numpy.reshape(dataset['setq'],(len(dataset['setq']),1))
    dataset_clean = numpy.concatenate((time_reshaped,set_v_reshaped,set_f_reshaped,set_q_reshaped),axis=1)
    # save to file
    numpy.savetxt(fcleanset,dataset_clean,fmt='%f,%f,%f,%f')
    # show the user the results of the processing job
    show_file_contents(fcleanset)
    timenow()
    print("Setpoint timecourse complete!\n")
    print("#######################\n")
    print("#######################\n\n")
    

def plot_temperature_timecourse(setpoint=None):
    ### THIS FUNCTION WORKS (BC 2016-10-01)
    fcleantemp = os.path.join(dout,'time_temperature.csv')
    # show the user the results of the processing job
    timenow()
    print('Plotting temperature data!')
    show_file_contents(fcleantemp)
    ##############################################
    ### IMPORTING THE DATA INTO xax, yax1, yax2
    ##############################################
    col_headers = ['Time','Tsurface','Ttube']
    col_types = (float,float,float)
    dataset = numpy.genfromtxt(fcleantemp,delimiter=',',
                               names=col_headers,
                               dtype=col_types)
    ##############################################
    ### PLOTTING THE DATA
    ##############################################
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(dataset['Time'],dataset['Tsurface'],label='Tsurface',
            linestyle='None',marker='o',markersize=1,color='red',markeredgecolor='red')
    ax.plot(dataset['Time'],dataset['Ttube'],label='Ttube',
            linestyle='None',marker='o',markersize=1,color='blue',markeredgecolor='blue',)
    if setpoint is not None:
        ax.axhline(setpoint,0,xax[-1],color='black',linestyle='--')
    ax.set_ylim([15,40])
    ax.set_xlim([0,13000])
    plt.title('He jet, PID @ 40C, 0.5 slpm, 10 kHz')
    ax.set_xlabel('runtime, seconds')
    ax.set_ylabel('temperature, degC')
    ax.legend(loc=0,markerscale=8)
    ax.grid(True)
    # save the plot to a file
    fplot = os.path.join(dplot,'temperature_history.png')
    fig.savefig(fplot,dpi=150)
    timenow()
    print('Temperature timecourse saved to {:s}'.format(fplot))
    print("#######################\n")
    print("#######################\n\n")
    plt.close(fig)

def plot_setpoint_timecourse(setpoint=None):
    ### THIS FUNCTION WORKS (BC 2016-10-01)
    fcleanset = os.path.join(dout,'time_setpoint.csv')
    # show the user the results of the processing job
    timenow()
    print("Plotting setpoint data!")
    show_file_contents(fcleanset)
    ##############################################
    ### IMPORTING THE DATA INTO `dataset
    ##############################################
    col_headers = ['Time','Set_v','Set_f','Set_q']
    col_types = (float,float,float,float)
    dataset = numpy.genfromtxt(fcleanset,delimiter=',',
                               names=col_headers,
                               dtype=col_types)
    ##############################################
    ### PLOTTING THE DATA
    ##############################################
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(dataset['Time'],dataset['Set_v'],label='Set_v',
            linestyle='None',marker='o',markersize=1,color='red',markeredgecolor='red')
    ax.plot(dataset['Time'],dataset['Set_f'],label='Set_f',
            linestyle='None',marker='o',markersize=1,color='blue',markeredgecolor='blue',)
    ax.plot(dataset['Time'],dataset['Set_q'],label='Set_q',
            linestyle='None',marker='o',markersize=1,color='green',markeredgecolor='green',)
    if setpoint is not None:
        ax.axhline(setpoint,0,xax[-1],color='black',linestyle='--')
    ax.set_ylim([0,100])
    ax.set_xlim([0,13000])
    plt.title('He jet, PID @ 40C, 0.5 slpm, 10 kHz')
    ax.set_xlabel('runtime, seconds')
    ax.set_ylabel('setpoint, %')
    ax.legend(loc=0,markerscale=8)
    ax.grid(True)
    # save the plot to a file
    fplot = os.path.join(dplot,'setpoint_history.png')
    fig.savefig(fplot,dpi=150)
    timenow()
    print('Setpoint timecourse saved to {:s}'.format(fplot))
    print("#######################\n")
    print("#######################\n\n")
    plt.close(fig)

def prep_power_data():
    """Prepares .csv files containing the time-power and voltage-power relationships"""
    ### THIS WORKS (BC 2016-10-01)
    timenow()
    print("Prepping power data...")
    fcleanpower = os.path.join(dout,'time_power.csv')
    fcleanpowervoltage = os.path.join(dout,'voltage_power.csv')
    
    powertime = []
    powervoltage = []
    
    # get the list of files in the oscilloscope directory
    files = sorted(os.listdir(dinosc))
    
    # for each of the oscilloscope traces..
    for f in files:
        filename = os.path.join(dinosc,f)
        # use the filename to get the time the datapoint was taken (relative to starttime)
        t = (datetime.strptime(f,"%Y-%m-%d_%H:%M:%S")-starttime).total_seconds()
        
        if (t > tmax):
                break
        
        # import the data in this file (time, channels 1-4)
        data = numpy.genfromtxt(filename,delimiter=',')
        xax = [line[0] for line in data]
        data_v = [line[1] for line in data]
        data_vcap = [line[2] for line in data]
        data_i = [line[3] for line in data]
        
        # grab the peak-to-peak voltage directly from the data
        voltage = max(data_v)-min(data_v)
        
        # calculate the power from Vapp and Vcap using the Lissajous method
        power = getpower(vapp=data_v,vcap=data_vcap,title=str(voltage).split('.')[0])
        
        # save the time-power and voltage-power relationships to an array
        powertime.append((t,power))
        powervoltage.append((voltage,power))
    # save the time-power and voltage-power relationships to a file
    numpy.savetxt(fcleanpower,powertime,fmt='%f,%f')
    numpy.savetxt(fcleanpowervoltage,powervoltage,fmt='%f,%f')
    
    # show the user the results of the processing job
    show_file_contents(fcleanpower)
    timenow()
    print("Power timecourse complete!\n\n")
    show_file_contents(fcleanpowervoltage)
    timenow()
    print("Power-voltage complete!\n")
    print("#######################\n")
    print("#######################\n\n")

def getpower(vapp,vcap,title):
    """Calculates the power, given the applied and Lissajous capacitor voltages"""
    vcap_value = 0.47 # 0.47 = 47nF 
    # iff >1, apply a window smoothing function to the voltage data
    windowSize = 1
    Liss = create_lissajous(Vapp=vapp,Vcap=vcap,window=windowSize)
    
    # find the area enclosed by the Lissajous figure of the first 1000 datapoints
    area = find_area(Liss[0:1000])
    # wattage is proportional to the area, the Lissajous cap size, sample duration (0.001s)
    watts = int(str(area).split('.')[0])*vcap_value/1000.
    
    # if specified in the header, plot each lissajous figure
    if plot_each_lissajous:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.scatter([p[0] for p in Liss], [p[1] for p in Liss])
        plt.title('The area of the Lissajous figure is: ' + str(area))
        ax.set_xlabel('volts, applied')
        ax.set_ylabel('volts, capacitor')
        fname = title + '_pwr' + str(area).split('.')[0] + '.png'
        fsave = os.path.join(dplot,'lissajous',fname)
        plt.savefig(fsave,dpi=150)
        plt.close(fig)
        
    # return the wattage measured at this timepoint
    return(watts)

def create_lissajous(Vapp,Vcap,window=1):
    """Generates a Lissajous figure from the applied and capacitor voltage oscilloscope traces."""
    #x=[Vapp[i][1] for i in range(0,len(Vapp))] # if your data also includes time
    #y=[Vcap[i][1] for i in range(0,len(Vcap))]
    x = Vapp
    y = Vcap
    if window!= 1:
        x = moving_average(x,window)
        y = moving_average(y,window)
    Liss = []
    for i in range(0,len(Vapp)):
        Liss.append((x[i],y[i]))
    return Liss

def moving_average(interval, window_size):
    """Finds the moving average of a dataset over a window size."""
    # algorithm via http://stackoverflow.com/questions/11352047/finding-moving-average-from-data-points-in-python
    window = numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

def find_area(array):
    """Find the array of a polygon defined as a set of Cartesian points in an array."""
    # algorithm via http://www.arachnoid.com/area_irregular_polygon/index.html
    a = 0
    ox,oy = array[0]
    for x,y in array[1:]:
        a += (x*oy-y*ox)
        ox,oy = x,y
    return abs(a/2)

def prep_power_setpoint():
    fcleanpower = os.path.join(dout,'time_power.csv') 
    fcleanset = os.path.join(dout,'time_setpoint.csv')
    fcleanpowerset = os.path.join(dout,'power_setpoint.csv')
    
    timenow()
    print("Determining power-setpoint relationships...")
    
    col_headers = ['Time','Power']
    col_types = [float,float]
    time_power = numpy.genfromtxt(fcleanpower,delimiter=',',
                                  names=col_headers,
                                  dtype=col_types)
    timenow()
    print("Time-power data loaded!")
    
    col_headers = ['Time','Set_v','Set_f','Set_q']
    col_types = (float,float,float,float)
    time_setpoint = numpy.genfromtxt(fcleanset,delimiter=',',
                                     names=col_headers,
                                     dtype=col_types)
    timenow()
    print("Time-setpoint data loaded!")
    
    power_setpoint = []
    with open(fcleanpowerset,'w') as myfile:
        for line in time_power:
            t = 0
            if (t > tmax):
                break
            t = line[0]
            p = line[1]
            set_v = numpy.interp(t,time_setpoint['Time'],time_setpoint['Set_v'])
            set_f = numpy.interp(t,time_setpoint['Time'],time_setpoint['Set_f'])
            set_q = numpy.interp(t,time_setpoint['Time'],time_setpoint['Set_q'])
            
            #power_setpoint.append((p,set_v,set_f,set_q))
            myfile.write('{:f},{:f},{:f},{:f},{:f}\n'.format(t,p,set_v,set_f,set_q))
    
    ## save the power-setpoint relationships to a file
    ## estimated time to completion with full dataset: ~40 minutes
    #numpy.savetxt(fcleanpowerset,power_setpoint,fmt='%f,%f,%f,%f')
    
    # show the user the results of the processing job
    show_file_contents(fcleanpowerset)
    timenow()
    print("Power-Setpoint .csv complete!\n")
    print("#######################\n")
    print("#######################\n\n")

def plot_power_setpoint(var='Set_v'):
    ### TESTING (BC 2016-10-01)
    fcleanpowerset = os.path.join(dout,'power_setpoint.csv')
    # show the user the results of the processing job
    timenow()
    print('Plotting power-setup data!')
    show_file_contents(fcleanpowerset)
    ##############################################
    ### IMPORTING THE DATA INTO `dataset`
    ##############################################
    col_headers = ['Time','Power','Set_v','Set_f','Set_q']
    col_types = (float,float,float,float,float)
    dataset = numpy.genfromtxt(fcleanpowerset,delimiter=',',
                               names=col_headers,
                               dtype=col_types)
    ##############################################
    ### PLOTTING THE DATA
    ##############################################
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(dataset[var],dataset['Power'],label='Power',
            linestyle='None',marker='o',markersize=1,color='red',markeredgecolor='red')
    #ax.set_ylim([15,40])
    #ax.set_xlim([0,13000])
    plt.title('He jet, PID @ 40C, 0.5 slpm, 10 kHz')
    ax.set_xlabel(var)
    ax.set_ylabel('Power, W')
    ax.legend(loc=0,markerscale=8)
    ax.grid(True)
    # save the plot to a file
    fplot = os.path.join(dplot,'{:s}-power.png'.format(var))
    fig.savefig(fplot,dpi=150)
    timenow()
    print('{:s} timecourse saved to {:s}'.format(var,fplot))
    print("#######################\n")
    print("#######################\n\n")
    plt.close(fig)

def plot_power_setpoint_subset(xvar='Set_q',varied='Set_v',fixed='Set_f',fixedval=60):
    ### TESTING (BC 2016-10-01)
    fcleanpowerset = os.path.join(dout,'power_setpoint.csv')
    # show the user the results of the processing job
    timenow()
    print('Plotting power-setup data!')
    show_file_contents(fcleanpowerset)
    ##############################################
    ### IMPORTING THE DATA INTO `dataset`
    ##############################################
    col_headers = ['Time','Power','Set_v','Set_f','Set_q']
    col_types = (float,float,float,float,float)
    dataset = numpy.genfromtxt(fcleanpowerset,delimiter=',',
                               names=col_headers,
                               dtype=col_types)
    t_index = 0
    p_index = 1
    q_index = 4
    v_index = 2
    f_index = 3
    q_vals = [3,5,7,9,11,13,15,17,19,21,23,25]
    v_vals = [40,45,50,55,60,65,70,75,80,85,90,95,100]
    f_vals = [0,10,20,30,40,50,60]
    
    if xvar == 'Set_q':
        xvar_index = q_index
        xvals = q_vals
    elif xvar == 'Set_v':
        xvar_index = v_index
        xvals = v_vals
    elif xvar == 'Set_f':
        xvar_index = f_index
        xvals = f_vals
    
    if varied == 'Set_q':
        varied_index = q_index
        varied_vals = q_vals
    elif varied == 'Set_v':
        varied_index = v_index
        varied_vals = v_vals
    elif varied == 'Set_f':
        varied_index = f_index
        varied_vals = f_vals
    
    if fixed =='Set_q':
        fixed_index = q_index
        fixed_vals = q_vals
    elif fixed =='Set_v':
        fixed_index = v_index
        fixed_vals = v_vals
    elif fixed =='Set_f':
        fixed_index = f_index
        fixed_vals = f_vals
    
    ##############################################
    ### PLOTTING THE DATA
    ##############################################
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # colormap stuff
    cm_start = 0.0
    cm_stop = 1.0
    cm_numlines= len(f_vals)
    colors = [ cm.viridis(x) for x in numpy.linspace(cm_start, cm_stop, cm_numlines) ]
    
    # plot particular series!
    for num,varied_val in enumerate(varied_vals):
        myX = []
        myY = []
        for line in dataset:
            if (line[varied_index] == varied_val) and (line[fixed_index] == fixedval):
                myX.append(0.1*line[xvar_index])
                myY.append(line[p_index])
        # plot this set!
        #ax.plot(myX,myY,label='{}={}'.format(varied,varied_val),color=colors[num],
        #    linestyle='None',marker='o',markersize=5,markeredgewidth=0)
        ax.plot(myX,myY,label='{0:.2f}kHz'.format(6.7+0.00159*(varied_val**2)),color=colors[num],
            linestyle='None',marker='o',markersize=5,markeredgewidth=0)
    
    myxvals = numpy.linspace(0,25,10)
    myyvals = lambda s: 10*s-4.5
    ax.plot(myxvals,myyvals(myxvals),linestyle='--',color='black',linewidth=3)
    ax.axhline(7.5,0,1,color='black',linestyle='--',linewidth=3)
    
    ax.set_ylim([0,15])
    ax.set_xlim([0,2.5])
    plt.title('He jet power vs gas flowrate, frequency')
    #ax.set_xlabel(xvar) #automatic
    ax.set_xlabel('gas flowrate,L/min')
    ax.set_ylabel('Power, W')
    ax.legend(loc=0,markerscale=1.5,fontsize=8,numpoints=3,ncol=2)
    ax.grid(True)
    # save the plot to a file
    fplot = os.path.join(dplot,'{:s}-power_varied_{:s}.png'.format(xvar,varied))
    fig.savefig(fplot,dpi=150)
    timenow()
    print('{:s} timecourse saved to {:s}'.format(xvar,fplot))
    print("#######################\n")
    print("#######################\n\n")
    plt.close(fig)

In [ ]:
plot_power_setpoint_subset(xvar='Set_q',varied='Set_f',fixed='Set_v',fixedval=nom_v)

In [ ]:
prep_arduino_data()
plot_temperature_timecourse()
plot_setpoint_timecourse()
prep_power_data()
prep_power_setpoint()
plot_power_setpoint('Set_v')
plot_power_setpoint('Set_f')
plot_power_setpoint('Set_q')

plot_power_setpoint_subset(xvar='Set_q',varied='Set_v',fixed='Set_f',fixedval=nom_f)
plot_power_setpoint_subset(xvar='Set_q',varied='Set_f',fixed='Set_v',fixedval=nom_v)
plot_power_setpoint_subset(xvar='Set_f',varied='Set_v',fixed='Set_q',fixedval=nom_q)
plot_power_setpoint_subset(xvar='Set_f',varied='Set_q',fixed='Set_v',fixedval=nom_v)
plot_power_setpoint_subset(xvar='Set_v',varied='Set_q',fixed='Set_f',fixedval=nom_f)
plot_power_setpoint_subset(xvar='Set_v',varied='Set_f',fixed='Set_q',fixedval=nom_q)

In [5]:
prep_arduino_data()

The time now is: 2016-12-16_14:01:32
Building temperature timecourse...
Now processing data file: /mnt/storage/data/2016-12-13_19:55:12/in/temperature/temperaturehistory
The contents of the file look like: 
    93,2820:00:03,2814595,0,0.00,0.00,0.00,0.00,22.86,22.46,0.00,0.00,0.00,1.98,0.00,1.00,0.00,494.00,2.00,24.90,

    2016-12-13_20:00:03,2814655,03,2814595,0,0.00,0.00,0.00,0.00,22.86,22.46,0.00,0.00,0.00,1.98,0.00,1.00,0.00,494.00,2.00,24.90,

    2016-12-13_20:00:03,2814655,0,0.00,0.00,0.00,0.00,22.82,22.46,0.00,0.00,0.00,1.98,0.00,1.00,0.00,498.00,5.00,24.90,

    2016-12-13_20:00:03,2814716,0,0.00,0.00,0.00,0.00,22.82,23.04,0.00,0.00,0.00,1.98,0.00,1.00,0.00,495.00,2.00,24.90,

    2016-12-13_20:00:03,2814777,0,0.00,0.00,0.00,0.00,22.82,23.04,0.00,0.00,0.00,1.98,0.00,1.00,0.00,494.00,3.00,24.90,



TypeError: can't concat bytes to float